In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from mrsimulator import Simulator, SpinSystem, Site
from mrsimulator import signal_processor as sp
from mrsimulator.method import SpectralDimension
from mrsimulator.method.lib import ThreeQ_VAS, SSB2D
from mrsimulator.spin_system.tensors import SymmetricTensor

import ase.io as ase_io

from soprano.properties import nmr

In [ ]:
def get_3qmas(frm, tag):
    x = frm.copy()
    x.arrays["ms"] = x.arrays[tag].reshape(-1, 3, 3)

    if "QM" in tag:
        x.arrays["efg"] = x.arrays["QM_efg"].reshape(-1, 3, 3)
    else:
        x.arrays["efg"] = x.arrays["ML_efg"].reshape(-1, 3, 3)
    ms_iso = nmr.MSIsotropy().get(x)
    ms_zeta = nmr.MSAnisotropy().get(x)
    ms_eta = nmr.MSAsymmetry().get(x)
    cq = nmr.EFGQuadrupolarConstant().get(x)
    efg_eta = nmr.EFGAsymmetry().get(x)
    O = [
        Site(
            isotope="17O",
            isotropic_chemical_shift=ms_iso[ii],  # in ppm
            shielding_symmetric=SymmetricTensor(
                zeta=ms_zeta[ii], eta=ms_eta[ii]
            ),  # zeta in ppm
            quadrupolar=SymmetricTensor(Cq=cq[ii], eta=efg_eta[ii]),
        )
        for ii in np.arange(len(x))[(x.numbers == 8)]
    ]
    spin_system = [SpinSystem(sites=[y]) for y in O]
    sim = Simulator(spin_systems=spin_system)
    method = ThreeQ_VAS(
        channels=["17O"],
        magnetic_flux_density=14.1,  # in T
        # rotor_frequency=10e3,
        spectral_dimensions=[
            SpectralDimension(
                count=1000,
                spectral_width=4e4,  # in Hz
                # reference_offset=-10.5e3,  # in Hz
                reference_offset=0,  # in Hz
                label="Isotropic dimension",
            ),
            # The last spectral dimension block is the direct-dimension
            SpectralDimension(
                count=1000,
                spectral_width=4.5e4,  # in Hz
                # reference_offset=2e4,  # in Hz
                reference_offset=0,
                label="MAS dimension",
            ),
        ],
    )
    sim.methods = [method]  # add the method.
    sim.run()  # Run the simulation

    dataset = sim.methods[0].simulation
    processor = sp.SignalProcessor(
        operations=[
            # Gaussian convolution along both dimensions.
            sp.IFFT(dim_index=(0, 1)),
            sp.apodization.Gaussian(FWHM="0.3 kHz", dim_index=0),
            sp.apodization.Gaussian(FWHM="0.15 kHz", dim_index=1),
            sp.FFT(dim_index=(0, 1)),
        ]
    )

    processed_dataset = processor.apply_operations(dataset=dataset)
    processed_dataset /= processed_dataset.max()

    return processed_dataset.real

In [ ]:
def get_pass(frm, tag):
    x = frm.copy()
    x.arrays["ms"] = x.arrays[tag].reshape(-1, 3, 3)
    ms_iso = nmr.MSIsotropy().get(x)
    ms_zeta = nmr.MSAnisotropy().get(x)
    ms_eta = nmr.MSAsymmetry().get(x)
    Si = [
        Site(
            isotope="29Si",
            isotropic_chemical_shift=ms_iso[ii],  # in ppm
            shielding_symmetric=SymmetricTensor(
                zeta=ms_zeta[ii], eta=ms_eta[ii]
            ),  # zeta in ppm
        )
        for ii in np.arange(len(x))[(x.numbers == 14)]
    ]
    spin_system = [SpinSystem(sites=[y]) for y in Si]
    sim = Simulator(spin_systems=spin_system)

    PASS = SSB2D(
        channels=["29Si"],
        magnetic_flux_density=9.7,
        rotor_frequency=1000,
        spectral_dimensions=[
            SpectralDimension(
                count=30 * 4,
                spectral_width=1000 * 30,  # value in Hz; count vs rotor frequency
                label="Anisotropic dimension",
            ),
            SpectralDimension(
                count=18192,
                spectral_width=1e5,  # value in Hz
                # reference_offset=1.85e4,  # value in Hz
                reference_offset=0,  # value in Hz
                label="Isotropic dimension",
            ),
        ],
    )
    sim.methods = [PASS]  # add the method.

    sim.config.number_of_sidebands = 8

    # run the simulation.
    sim.run()

    dataset = sim.methods[0].simulation
    processor = sp.SignalProcessor(
        operations=[
            sp.IFFT(dim_index=0),
            sp.apodization.Exponential(FWHM="100 Hz", dim_index=0),
            sp.FFT(dim_index=0),
        ]
    )
    processed_dataset = processor.apply_operations(dataset=dataset).real
    processed_dataset /= processed_dataset.max()

    return processed_dataset.real

In [ ]:
frames = ase_io.read(
    "../data/hypothetical_zeolites/hypozeo_ml_dft_isd_tp_ms_efg.xyz", ":"
)

In [ ]:
list(frames[0].arrays.keys())

In [ ]:
i_3qmas = {}
t_3qmas = {}
r_3qmas = {}

for i in [26, 7]:
    i_3qmas[i] = get_3qmas(frames[i], tag="ML_ISD_ms")
    t_3qmas[i] = get_3qmas(frames[i], tag="ML_TP_ms")
    r_3qmas[i] = get_3qmas(frames[i], tag="QM_ms")

In [ ]:
i_pass = {}
t_pass = {}
r_pass = {}

for i in [14, 25]:
    i_pass[i] = get_pass(frames[i], tag="ML_ISD_ms")
    t_pass[i] = get_pass(frames[i], tag="ML_TP_ms")
    r_pass[i] = get_pass(frames[i], tag="QM_ms")

In [ ]:
fig = plt.figure(figsize=(3.5, 4), layout="constrained")
# num_lev = 4

levels = [0.03, 0.1, 0.5, 0.9, 0.97]

ax = fig.add_subplot(221, projection="csdm")
ii = 25
prop = i_pass[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C1", **options, linestyles="solid")

prop = t_pass[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C0", **options, linestyles="solid")

prop = r_pass[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="k", **options, linestyles="dashed")

ax.set_xlabel("")
ax.set_ylabel("")
ax.set_ylim(30, -30)
ax.set_xlim(465, 435)
ax.set_xticklabels(())
ax.set_yticks((-20, 0, 20))
ax.tick_params(axis="both", labelsize=9)

ax.tick_params(axis="both", labelsize=9)

###############################################
###############################################
###############################################

ax = fig.add_subplot(223, projection="csdm")
ii = 14
prop = i_pass[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C1", **options, linestyles="solid")

prop = t_pass[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C0", **options, linestyles="solid")

prop = r_pass[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="k", **options, linestyles="dashed")

ax.set_xlabel("")
ax.set_ylabel("")
ax.set_ylim(30, -30)
ax.set_xlim(465, 435)

ax.set_yticks((-20, 0, 20))
ax.tick_params(axis="both", labelsize=9)
fig.supylabel("$^{29}$Si anisotropic dimension (ppm)", fontsize=9)
ax.set_xlabel("$^{29}$Si isotropic dimension (ppm)", fontsize=9)

ax.tick_params(axis="both", labelsize=9)

###############################################
###############################################
###############################################

ax = fig.add_subplot(222, projection="csdm")
ii = 7
prop = i_3qmas[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C1", **options, linestyles="solid")

prop = t_3qmas[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C0", **options, linestyles="solid")

prop = r_3qmas[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="k", **options, linestyles="dashed")

ax.set_xlabel("")
ax.set_ylabel("")
ax.set_ylim(-112, -145)
ax.set_xlim(235, 110)


ax.set_xticklabels(())
ax.tick_params(axis="both", labelsize=9)

ax.tick_params(axis="both", labelsize=9)


###############################################
###############################################
###############################################

ax = fig.add_subplot(224, projection="csdm")
ii = 26
prop = i_3qmas[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C1", **options, linestyles="solid")

prop = t_3qmas[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="C0", **options, linestyles="solid")

prop = r_3qmas[ii]
max_amp = prop.max()
options = dict(levels=levels, alpha=1, linewidths=0.5)  # plot options
ax.contour(prop, colors="k", **options, linestyles="dashed")

ax.set_xlabel("")
ax.set_ylabel("")
ax.set_ylim(-112, -145)
ax.set_xlim(235, 110)


ax.tick_params(axis="both", labelsize=9)
ax.set_ylabel("$^{17}$O isotropic dimension (ppm)", fontsize=9)
ax.set_xlabel("$^{17}$O MAS dimension (ppm)", fontsize=9)

ax.tick_params(axis="both", labelsize=9)

lines = [
    Line2D([0], [0], label="ML ISD", color="C1", ls="solid", lw=0.75),
    Line2D([0], [0], label="ML TP", color="C0", ls="solid", lw=0.75),
    Line2D([0], [0], label="QM", color="k", ls="dashed", lw=0.75),
]
fig.legend(
    handles=lines, fontsize=9, bbox_to_anchor=(0.975, 1.07), ncols=3, frameon=False
)
# fig.savefig("./hypozeo_preds_o_si_best_worst_corr_v8.svg", dpi=300, bbox_inches="tight")